The global schema for our application looks like- 
Customer (Name, email, phone#, loyaltyPts, Age Group, medical condition (optional))
Flight (flight#, model, economy seats,business seats, first class seats, price-eco,price-business,price- first class availability, status, departure_time, landing_time, source, destination, distance, delay)

We will be performing schema matching to identify the fields that match with our global schema. We wil be doing this through linguistic matching primarily


Importing schemas from databases

In [1]:
import psycopg2

In [2]:
!pip install rapidfuzz

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   -------------- ------------------------- 0.6/1.6 MB 17.9 MB/s eta 0:00:01
   ---------------------------------------  1.6/1.6 MB 20.2 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 17.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from rapidfuzz import fuzz

def fuzzy_match(word, word_list):
    # Calculate similarity scores for each word in the list
    similarities = {w: fuzz.ratio(word, w) for w in word_list}
    return similarities

# Example usage

input_word = "king"
words_list = ["queen", "man", "woman", "child", "throne"]

# Perform fuzzy matchingresults = fuzzy_match(input_word, words_list)


# Print results
for word, similarity in results.items():
    print(f"Fuzzy similarity between '{input_word}' and '{word}': {similarity}")






Fuzzy similarity between 'king' and 'queen': 22.22222222222222
Fuzzy similarity between 'king' and 'man': 28.57142857142857
Fuzzy similarity between 'king' and 'woman': 22.22222222222222
Fuzzy similarity between 'king' and 'child': 22.22222222222222
Fuzzy similarity between 'king' and 'throne': 19.999999999999996


In [6]:
results

{'queen': 22.22222222222222,
 'man': 28.57142857142857,
 'woman': 22.22222222222222,
 'child': 22.22222222222222,
 'throne': 19.999999999999996}

In [8]:
Customer = [
    "Name", "Email", "Phone#", "LoyaltyPts", 
    "Age Group", "Medical Condition (optional)", 
    "City", "Address"
]

Flight = [
    "Flight#", "Model", "Economy-Seats", "Business_Seats", "First Class-Seats", 
    "Economy Price", "Business Price", "First Class Price", 
    "Status", "Departure_Time", "Landing_Time", 
    "Source", "Destination", "Distance", "Delay"
]
global_schema = {
    "Customer": Customer,
    "Flight": Flight
}

#############################################

# Airline I
Customer_Info = ["CustomerID", "FullName", "EmailAddress", "ContactNumber", "LoyaltyPoints"]
Customer_Age = ["CustomerID", "AgeGroup"]  # Example: "18-25", "26-35", etc.
Customer_Health = ["CustomerID", "HasMedicalCondition"]  # Binary yes/no
Flight_Info = [
    "Flight_id", "FlightCode", "AircraftModel", "TotalSeats", 
    "TicketPrice-Economy", "TicketPrice-Business", "TicketPrice-First Class", 
    "AvailabilityStatus-Economy", "AvailabilityStatus-Business", "AvailabilityStatus-First Class", 
    "DepartureTime", "ArrivalTime"
]
Flight_Route = [
    "Flight_id", "FlightCode", "DepartureAirport", "DestinationAirport", 
    "FlightDistance", "CurrentStatus", "AverageDelay"
]

# Airline II
FlightDetails = [
    "FlightId", "FlightNumber", "AircraftType", "TotalCapacity", 
    "PricePerSeat-Economy", "PricePerSeat-Business", 
    "AvailableSeats-Economy", "AvailableSeats-Business", 
    "FlightStatus", "TakeoffTime", "LandTime"
]
Route = ["FlightId", "Origin", "Destination", "DistanceTraveled", "EstimatedDelay"]
Passenger = ["PassengerName", "Email", "Contact", "FrequentFlyerPoints", "DateOfBirth", "City"]

# Airline III
Clients = ["Full_Name", "EmailID", "ContactNo", "Age", "PreExistingCondition"]
Loyalty = ["ClientEmail", "Points"]
FlightSchedule = [
    "FlightID", "Flight code", "SeatsLeft", "FlightFare", 
    "Status", "StartTime", "EndTime"
]
RoutePlan = ["FlightID", "FromLocation", "ToLocation", "DistanceKM", "AvgDelay"]

# Consolidated view (if needed)
airline_schemas = {
    "Airline I": {
        "Customer_Info": Customer_Info,
        "Customer_Age": Customer_Age,
        "Customer_Health": Customer_Health,
        "Flight_Info": Flight_Info,
        "Flight_Route": Flight_Route
    },
    "Airline II": {
        "FlightDetails": FlightDetails,
        "Route": Route,
        "Passenger": Passenger
    },
    "Airline III": {
        "Clients": Clients,
        "Loyalty": Loyalty,
        "FlightSchedule": FlightSchedule,
        "RoutePlan": RoutePlan
    }
}


airline_schemas2=airline_schemas





In [11]:
mapping={}

for airlines in airline_schemas.values():
    all_fields=[]
    for fields in airlines.values():
        all_fields.extend(fields)
    for k,v in global_schema.items():
        for field in v:
            result = fuzzy_match(field, all_fields)
            best_match=""
            best_score=0
            for word, score in result.items():
                if score>best_score:
                    best_match=word
                    best_score=score
            if field not in mapping:
                mapping[field]=[best_match]
               
            
            else:
                mapping[field].append(best_match)


            
        




In [12]:
mapping

{'Name': ['FullName', 'LandTime', 'Full_Name'],
 'Email': ['EmailAddress', 'Email', 'EmailID'],
 'Phone#': ['FlightCode', 'PassengerName', 'Points'],
 'LoyaltyPts': ['LoyaltyPoints', 'TotalCapacity', 'ToLocation'],
 'Age Group': ['AgeGroup', 'FlightNumber', 'Age'],
 'Medical Condition (optional)': ['HasMedicalCondition',
  'Destination',
  'PreExistingCondition'],
 'City': ['Flight_id', 'City', 'ClientEmail'],
 'Address': ['EmailAddress', 'AircraftType', 'Age'],
 'Flight#': ['Flight_id', 'FlightId', 'FlightID'],
 'Model': ['AircraftModel', 'EstimatedDelay', 'Flight code'],
 'Economy-Seats': ['TotalSeats', 'PricePerSeat-Economy', 'SeatsLeft'],
 'Business_Seats': ['TicketPrice-Business',
  'PricePerSeat-Business',
  'SeatsLeft'],
 'First Class-Seats': ['TicketPrice-First Class', 'FlightStatus', 'SeatsLeft'],
 'Economy Price': ['TicketPrice-Economy', 'PricePerSeat-Economy', 'EndTime'],
 'Business Price': ['TicketPrice-Business',
  'PricePerSeat-Business',
  'DistanceKM'],
 'First Class Pr